In [10]:
import json
import numpy as np
import pandas as pd
with open('../results/rbf/simu_B_w_50_100_2000_AEPW1.json', 'r') as f:
    data_str = f.read()

results = json.loads(data_str)
print(results)
c_upper = []
kernel_ate_estimates = []
for c_uppers, models in results.items():
    for model in models:
        c_upper.append(float(c_uppers))
        kernel_ate_estimates.append(float(model[0]))

data = pd.DataFrame({'c_upper': c_upper, 'ATE': kernel_ate_estimates})

# 分组
grouped_data = data.groupby('c_upper')['ATE'].apply(list).tolist()

# 计算每个分组的平均值和方差
grouped_stats = []
for group in grouped_data:
    mean = np.mean(group)
    variance = np.var(group, ddof=1)  # 使用样本方差
    grouped_stats.append({'mean': mean, 'variance': variance})

# 打印结果
for i, stats in enumerate(grouped_stats):
    print(f"pi: {i/10}, Mean: {stats['mean']}, Variance: {stats['variance']}")

{'0.0': [[7.828710378078796, 0.57542394368472, 1681.9545550842088, 764.1733682929356, {'1': 1.2042318836465957, '-1': 1}, 0.6857064948534104], [5.445123899588054, 0.558664516121939, 1668.593549245104, 1000.0, {'1': 1.2665723945936402, '-1': 1}, 0.7029088131351134], [5.325219404260565, 0.5618352170833606, 1688.2692750118895, 1000.0, {'1': 1.2309944510056021, '-1': 1}, 0.658760948274676], [6.7698579798710625, 0.6016155968327638, 1682.949998613102, 1000.0, {'1': 1.133354291027725, '-1': 1}, 0.6824162301841522], [9.691381730868613, 0.6379102389528242, 1663.819507845622, 1000.0, {'1': 1.0984408084357389, '-1': 1}, 0.6609803364422497], [11.496519340876091, 0.6293409929542836, 1708.104186395783, 844.8427976103201, {'1': 1.0934549439100807, '-1': 1}, 0.6631176813702419], [4.4805579513104306, 0.6107162382342654, 1677.7182047498322, 1000.0, {'1': 1.3157056323430272, '-1': 1}, 0.6818250541629725], [3.997331915714085, 0.521291580412258, 1664.8441006416688, 1000.0, {'1': 1.4353207888661395, '-1': 1

In [16]:
num_datasets = 100

from sklearn.svm import SVC

def compute_wnayman(Y_obs, W, X, alpha, support_indices):
    
    # 计算权重
    weights = np.zeros(X.shape[0])
    weights[support_indices] = np.abs(alpha)
    lambda_i = weights / np.sum(weights)
    
    # 计算mu_c(x)和mu_t(x)，这里需要根据具体数据进行定义
    mu_c = np.mean(Y_obs[W == -1])
    mu_t = np.mean(Y_obs[W == 1])

    # 计算sigma_c^2(x)和sigma_t^2(x)，这里需要根据具体数据进行定义
    sigma_c_squared = np.var(Y_obs[W == -1])
    sigma_t_squared = np.var(Y_obs[W == 1])

    # 计算mu_i和sigma_i^2
    mu_i = np.where(W == -1, mu_c, mu_t)
    sigma_i_squared = np.where(W == -1, sigma_c_squared, sigma_t_squared)

    N_t = np.sum(W == 1)
    N_c = np.sum(W == -1)

    # 计算条件抽样方差
    conditional_variance = np.sum(lambda_i[W == 1]**2 * sigma_i_squared[W == 1]) + np.sum(lambda_i[W == -1]**2 * sigma_i_squared[W == -1])
    
    stderr = np.sqrt(conditional_variance)


    print("条件抽样方差:", conditional_variance)
    return stderr


for n in range(num_datasets):
    for c_uppers, models in results.items():
            simulate_data_B = pd.read_csv('../data_A_2000/data_scenario_G_n_2000_dataset_{}.csv'.format(n))
            X = simulate_data_B.drop(columns=['T', 'Y'])
            T = simulate_data_B['T']
            Y = simulate_data_B['Y']

            W = 2 * T - 1

            C = models[n][3]
            print(C)
            #class_weight = {1: best_params[1], -1: best_params[2]}

            best_model = SVC(C=C, kernel='rbf')
            best_model.fit(X, W)
            support_indices = best_model.support_
            alpha = np.abs(best_model.dual_coef_[0])
            STD = compute_wnayman(Y, W, X, alpha, support_indices)

            models[n][4] = STD


            print("STD:", STD)



3.526233040753632
条件抽样方差: 9.582860529458445e-05
STD: 0.009789208614315278
3.526233040753632
条件抽样方差: 9.582860529458445e-05
STD: 0.009789208614315278
3.526233040753632
条件抽样方差: 9.582860529458445e-05
STD: 0.009789208614315278
3.526233040753632
条件抽样方差: 9.582860529458445e-05
STD: 0.009789208614315278
3.1634996145739143
条件抽样方差: 9.578277792350819e-05
STD: 0.009786867625727252
3.1634996145739143
条件抽样方差: 9.578277792350819e-05
STD: 0.009786867625727252
3.1634996145739143
条件抽样方差: 9.578277792350819e-05
STD: 0.009786867625727252
3.1634996145739143
条件抽样方差: 9.578277792350819e-05
STD: 0.009786867625727252
286.97033315125236
条件抽样方差: 0.00019069254561613635
STD: 0.01380914717193413
286.97033315125236
条件抽样方差: 0.00019069254561613635
STD: 0.01380914717193413
286.97033315125236
条件抽样方差: 0.00019069254561613635
STD: 0.01380914717193413
286.97033315125236
条件抽样方差: 0.00019069254561613635
STD: 0.01380914717193413
0.5495304734442168
条件抽样方差: 9.045700538972589e-05
STD: 0.00951088878022059
0.5495304734442168
条件抽样方差: 9.0

In [2]:
import json
import numpy as np
import pandas as pd
with open('../results/rbf/simu_B_50_100_2000.json', 'r') as f:
    data_str = f.read()

results = json.loads(data_str)

num_datasets = 100

from sklearn.svm import SVC

def estimate_diff(Y, W, X, alpha, support_indices):

    # 计算权重
    weights = np.zeros(X.shape[0])
    weights[support_indices] = np.abs(alpha)
    alpha = weights / np.sum(weights)
    
    treated_mean = np.sum(Y[W == 1] * alpha[W == 1]) / np.sum(alpha[W == 1])
    control_mean = np.sum(Y[W == -1] * alpha[W == -1]) / np.sum(alpha[W == -1])
    
    treated_std = np.std(Y[W == 1] * alpha[W == 1], ddof=1)
    control_std = np.std(Y[W == -1] * alpha[W == -1], ddof=1)

    n_t = np.sum(W == 1)
    n_c = np.sum(W == -1)

    std_pool = np.sqrt(((n_t - 1)*treated_std**2 + (n_c - 1)*control_std**2) / (n_t + n_c -2))
    normed_diff = np.abs(treated_mean - control_mean) / std_pool

    return normed_diff

for n in range(num_datasets):
    for c_uppers, models in results.items():
            simulate_data_B = pd.read_csv('../data_A_2000/data_scenario_G_n_2000_dataset_{}.csv'.format(n))
            X = simulate_data_B.drop(columns=['T', 'Y'])
            T = simulate_data_B['T']
            Y = simulate_data_B['Y']

            W = 2 * T - 1

            C = models[n][1]
            print(C)
            #class_weight = {1: best_params[1], -1: best_params[2]}

            best_model = SVC(C=C, kernel='rbf')
            best_model.fit(X, W)
            support_indices = best_model.support_
            alpha = np.abs(best_model.dual_coef_[0])
            DIM = estimate_diff(Y, W, X, alpha, support_indices)

            models[n][4] = DIM


            print("DIM:", DIM)

0.57542394368472
DIM: 1933.600587096958
0.57542394368472
DIM: 1933.600587096958
0.57542394368472
DIM: 1933.600587096958
0.57542394368472
DIM: 1933.600587096958
0.57542394368472
DIM: 1933.600587096958
0.57542394368472
DIM: 1933.600587096958
0.57542394368472
DIM: 1933.600587096958
0.558664516121939
DIM: 1936.459084926747
0.558664516121939
DIM: 1936.459084926747
0.558664516121939
DIM: 1936.459084926747
0.558664516121939
DIM: 1936.459084926747
0.558664516121939
DIM: 1936.459084926747
0.558664516121939
DIM: 1936.459084926747
0.558664516121939
DIM: 1936.459084926747
0.5618352170833606
DIM: 1796.470957366407
0.5618352170833606
DIM: 1796.470957366407
0.5618352170833606
DIM: 1796.470957366407
0.5618352170833606
DIM: 1796.470957366407
0.5618352170833606
DIM: 1796.470957366407
0.5618352170833606
DIM: 1796.470957366407
0.5618352170833606
DIM: 1796.470957366407
0.6016155968327638
DIM: 1827.6362601136818
0.6016155968327638
DIM: 1827.6362601136818
0.6016155968327638
DIM: 1827.6362601136818
0.60161559

KeyboardInterrupt: 

In [5]:
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import SVC

def compute_wnayman(Y_obs, W, X, alpha, support_indices):
    # 计算权重
    weights = np.zeros(X.shape[0])
    weights[support_indices] = np.abs(alpha)
    lambda_i = weights / np.sum(weights)
    
    # 匹配机制：为每个处理组单位找到最相似的对照组单位
    control_indices = np.where(W == -1)[0]
    treated_indices = np.where(W == 1)[0]

    print(control_indices)
    print(treated_indices)

    # 使用K近邻匹配
    knn = NearestNeighbors(n_neighbors=1)
    X_array = X.to_numpy() if isinstance(X, pd.DataFrame) else X
    print(X_array)
    knn.fit(X_array[control_indices].reshape(-1, 1))
    distances, indices = knn.kneighbors(X_array[treated_indices].reshape(-1, 1))

    # 确保indices中的索引在control_indices的范围内
    indices = indices.flatten()

    print(indices)
    matched_control_indices = control_indices[indices]

    print(indices)

    # 计算残差
    epsilon_hat = np.zeros_like(Y_obs)
    epsilon_hat[treated_indices] = Y_obs[treated_indices] - Y_obs[matched_control_indices]

    # 计算调整后的方差
    sigma2_adj = np.zeros_like(Y_obs)
    sigma2_adj[treated_indices] = (epsilon_hat[treated_indices] - epsilon_hat[control_indices[indices.flatten()]]) ** 2 / 2

    # 计算抽样方差
    N_t = np.sum(W == 1)
    N_c = np.sum(W == 0)

    variance_treated = np.sum(lambda_i[treated_indices] ** 2 * sigma2_adj[treated_indices]) / N_t ** 2
    variance_control = np.sum(lambda_i[control_indices] ** 2 * sigma2_adj[control_indices]) / N_c ** 2

    total_variance = variance_treated + variance_control

    print(f"调整后的抽样方差估计为: {total_variance}")
    return np.sqrt(total_variance)


num_datasets = 100

for n in range(num_datasets):
    for c_uppers, models in results.items():
            simulate_data_B = pd.read_csv('../data_A_2000/data_scenario_G_n_2000_dataset_{}.csv'.format(n))
            X = simulate_data_B.drop(columns=['T', 'Y'])
            T = simulate_data_B['T']
            Y = simulate_data_B['Y']

            W = 2 * T - 1

            C = models[n][3]
            print(C)
            #class_weight = {1: best_params[1], -1: best_params[2]}

            best_model = SVC(C=C, kernel='rbf')
            best_model.fit(X, W)
            support_indices = best_model.support_
            alpha = np.abs(best_model.dual_coef_[0])
            STD = compute_wnayman(Y, W, X, alpha, support_indices)

            models[n][4] = STD


            print("STD:", STD)




4673.440869370874
[   0    3    7    8   10   14   17   18   20   24   27   30   31   36
   37   38   42   47   55   59   60   63   65   66   67   68   70   75
   78   79   86   88   98  100  101  102  103  106  108  111  114  116
  126  132  136  137  146  147  151  152  154  156  158  162  164  165
  167  168  171  175  176  177  178  180  183  184  185  192  193  195
  199  200  203  204  206  209  210  211  215  218  222  223  225  226
  227  228  230  231  233  236  238  239  242  246  250  252  253  255
  258  259  261  262  265  267  270  271  272  274  275  280  281  285
  295  297  298  299  302  304  305  306  307  309  313  315  318  319
  322  324  326  336  337  342  343  345  346  349  356  358  366  369
  370  371  373  378  382  384  386  387  388  393  395  396  397  399
  406  408  413  414  415  417  420  421  423  424  429  430  431  432
  433  434  437  438  440  441  445  446  447  448  451  452  454  457
  459  460  461  464  465  468  470  472  473  474  476  48

IndexError: index 7740 is out of bounds for axis 0 with size 836

In [17]:
print(results)

{'1000.0': [[-0.40041930255373376, 0.9849760010667652, 1543.58073342833, 3.526233040753632, 0.009789208614315278], [-0.401298787362557, 0.860377086159801, 1618.726973380411, 3.1634996145739143, 0.009786867625727252], [-0.410606894767629, 0.9169275743647632, 754.8879032040497, 286.97033315125236, 0.01380914717193413], [-0.40013213931502434, 0.9451735855027152, 1680.5731894450955, 0.5495304734442168, 0.00951088878022059], [-0.3925634021637755, 0.9659276715183227, 1652.1023476439907, 0.5902244248813002, 0.009564964004387565], [-0.4006709266700024, 0.9005551801954697, 1575.379740106935, 2.3135232502381924, 0.009791476479191555], [-0.39398970877525624, 0.9313311828930723, 1645.8234925199558, 0.019232844888462216, 0.009349230135606026], [-0.40221591233515386, 0.9692065158518245, 1693.1009555676228, 0.3918587921595072, 0.009281499353604592], [-0.4032192655869396, 0.9264030300183895, 1691.9961251529398, 0.019232844888462216, 0.009428717738009863], [-0.39554517513175275, 0.8596317654367235, 167

In [18]:
class NumpyEncoder(json.JSONEncoder):
    """自定义JSON编码器，用于处理numpy数据类型"""
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)
        
data = results

data_str = json.dumps(data, cls=NumpyEncoder, indent=4)


with open(f'../results/rbf/simu_A_30_100_2000.json', 'w') as f:
    f.write(data_str)